In [25]:
#import import_ipynb
#from model import *
import numpy as np
import itertools
import glob
import os
import json
import re
import pickle
import math
import tensorflow as tf
from tensorflow import keras
from keras.models import Model
from keras.backend import mean, sum, expand_dims, tanh
from keras.layers import Dense, Input, Dropout, LSTM, Activation, GRU, Concatenate, Multiply, Reshape, RepeatVector, Dot, Softmax
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.initializers import glorot_uniform
from keras.layers import Lambda
from keras.optimizers import Adam
np.random.seed(1)
from keras.models import Sequential


In [26]:
# given x (?,dim) always return (max_len, dim) by padding with zeros
def padding(max_len, x, dim ):
    x_len = x.shape[0]
    # truncate if more than max_len else pad with zeros
    if(x_len >= max_len):
        return x[0:max_len]
    else:   
        padding = np.zeros((max_len-x_len, dim))     
        if(len(x.shape) > 1):
            assert dim == x.shape[1],"dimensions didnt match in padding"   
            return np.concatenate((x, padding))
        else:
            return padding
        
# given 1D array x, we return array of shape (max_len, dim) by replicating each element dim times and apdding with zeros at end
def reshape_mask(max_len, x, dim):
    # mask is a 1D array so padd with dim == 1
    padded_question_mask = np.squeeze(padding(max_len, x, 1))
    final_question_mask = []
    for qm in padded_question_mask:
        # make each mask index into array of dimension dim
        final_question_mask.append(np.array([qm]*dim))
    return np.asarray(final_question_mask)


In [27]:
s_DimOfQuestionVector = 300
s_DimOfTripletVector  = 900
s_DimOfMask           = 50
s_QuestionMaxWords    = 60
s_TripletsMaxNumbers  = 500

In [28]:
def prepare_data(data, 
                 question_max_words, 
                 triplets_max_numbers, 
                 dimOfQuestionVector, 
                 dimOfTripletVector, 
                 dimOfMask):
    question_vectors = []
    question_masks = []
    triplet_vectors = []
    Y = []
    for elem in data:
        item_question             = elem["question"]
        padded_item_question      = padding(question_max_words, item_question, dimOfQuestionVector)
        item_question_mask        = np.reshape(elem["question_mask"], (len(elem["question_mask"]), 1) )
        padded_item_question_mask = reshape_mask(question_max_words, item_question_mask, dimOfMask)
        item_correct_triplets     = elem["correct_triplets"]
        item_wrong_triplets       = elem["wrong_triplets"]
        
        # for triplets take mean of each S, P  and 0 to get 3,300 vectors which are then concatenated to dimOfTripletVector vector
        item_correct_triplets_vectors = [] 
        for tr in item_correct_triplets: # There are triplets_max_numbers triplets
            # Each tr has s, p and o. Each of the _s, _p and _o's are (num_tokens, dimOfTripletVector) size
            _s = np.mean(tr[0], axis=0)
            _p = np.mean(tr[1], axis=0)
            _o = np.mean(tr[2], axis=0)
            item_correct_triplets_vectors.append(np.concatenate((_s, _p, _o))) # adding (900,) vector for each triplet
        #pad triplet vectors so we have (triplets_max_numbers, dimOfTripletVector)  encoding for all triplets
        padded_item_triplets_vectors = padding(triplets_max_numbers, np.asarray(item_correct_triplets_vectors), dimOfTripletVector)
        
        triplet_vectors.append(padded_item_triplets_vectors)
        question_vectors.append(padded_item_question)
        question_masks.append(padded_item_question_mask)
        Y.append(1)

        # for wrong triplets
        item_wrong_triplets_vectors = []
        for tr in item_wrong_triplets:
            _s = np.mean(tr[0], axis=0)
            _p = np.mean(tr[1], axis=0)
            _o = np.mean(tr[2], axis=0)
            item_wrong_triplets_vectors.append(np.concatenate((_s, _p, _o)))
        padded_item_wrong_triplets_vectors = padding(triplets_max_numbers, np.asarray(item_wrong_triplets_vectors), dimOfTripletVector)
                
        triplet_vectors.append(padded_item_wrong_triplets_vectors)
        question_vectors.append(padded_item_question)
        question_masks.append(padded_item_question_mask)
        Y.append(0)
        
        
        
    question_vectors = np.asarray(question_vectors)
    question_masks   = np.asarray(question_masks)
    triplet_vectors  = np.asarray(triplet_vectors)
    Y                = np.asarray(Y)
    return question_vectors, question_masks, triplet_vectors, Y

def generate_model_inputs():
    file_list = glob.glob('X:/cs230/proj0/ned-graphs/data_wiki_encoded/train/train_8.ipwk')
    data = []
    num = 0
    for inputFile in file_list:
        print(inputFile)
        f = open(inputFile, 'rb')
        test = pickle.load(f)
        data = np.append(data, test)
        f.close()
        num += 1
        if num == 1:
            break
    
    return prepare_data(data, 
                        question_max_words   = s_QuestionMaxWords, 
                        triplets_max_numbers = s_TripletsMaxNumbers, 
                        dimOfQuestionVector  = s_DimOfQuestionVector, 
                        dimOfTripletVector   = s_DimOfTripletVector, 
                        dimOfMask            = s_DimOfMask )

question_vectors, question_masks, triplet_vectors, Y = generate_model_inputs()#prepare_data(data, question_max_words=60, triplets_max_numbers=500)

X:/cs230/proj0/ned-graphs/data_wiki_encoded/train/train_8.ipwk


In [29]:
print(question_vectors.shape)
print(question_masks.shape)
print(triplet_vectors.shape)
print(Y.shape)

(200, 60, 300)
(200, 60, 50)
(200, 500, 900)
(200,)


In [30]:
questions_input_shape      = (s_QuestionMaxWords,   s_DimOfQuestionVector)
questions_mask_input_shape = (s_QuestionMaxWords,   s_DimOfMask )
triplets_input_shape       = (s_TripletsMaxNumbers, s_DimOfTripletVector )
# question_vectors(, 60, 300), question_masks(, 60, 300), triplet_vectors (,500,900), Y

gru_dimension = s_DimOfMask

In [31]:
# question inputs
question_vectors_input = Input(shape=questions_input_shape,      dtype='float32', name="question_vectors_input")   
question_masks_input   = Input(shape=questions_mask_input_shape, dtype='float32', name="question_masks_input")

# triplets input
triplets_input = Input(shape=triplets_input_shape, dtype='float32', name="triplets_input")  #(None, 500, 900)

# layers
# gru1 = Bidirectional(GRU( 50, return_sequences=True, name="Sentence_GRU1"))
# gru2 = Bidirectional(GRU( 50, return_sequences=False, name="Sentence_GRU2"))
# tgru1 = Bidirectional(GRU( 50, return_sequences=True, name="Triplet_GRU1"))
# tgru2 = Bidirectional(GRU( 50, return_sequences=False, name="Triplet_GRU2"))

gru1 =  (GRU( gru_dimension,  return_sequences=True,  name="Sentence_GRU1"))
gru2 =  (GRU( gru_dimension,  return_sequences=True,  name="Sentence_GRU2"))
tgru1 = (GRU( gru_dimension,  return_sequences=True,  name="Triplet_GRU1"))
tgru2 = (GRU( gru_dimension,  return_sequences=True, name="Triplet_GRU2"))

mean_layer = Lambda(lambda xin: mean(xin, axis=1), name="SentenceMean")

repeat_layer       = RepeatVector(triplets_input_shape[0], name="AttentionRepeat")
sum_layer          = Lambda(lambda xin: sum(xin, axis=-1), name="AttentionDotsum")
soft_max_layer     = Softmax(name="AttentionSofMax")
weighted_sum_layer = Lambda(lambda X: sum(X[0]*expand_dims(X[1],axis=-1), axis=1), name="AttentionWeightedSum")

X_question = gru2(gru1(question_vectors_input))             # (None, 3000, 50) -> (None, 60, 50)
X_question = Multiply()([question_masks_input, X_question]) # (None, 60, 50)
X_question = mean_layer(X_question)                         # (None, 60, 50) -> (None, 50)

X_triplets = Dense(gru_dimension, activation='relu', name="Dense50T")(triplets_input)    # (None,500,900) -> (None,500,50)
X_triplets = tgru2(tgru1(X_triplets))                                         # (None,500,50)  -> (None,500,50)

X_attention = repeat_layer(X_question)
# compute Wa*X_triplets
X_triplets_wegithed = Dense(gru_dimension, use_bias=False, name="AttentionDense")(X_triplets)
# compute dot of (Wa*X_triplets) with X_question
X_attention = Multiply(name="AttentionDotMul")([X_triplets_wegithed, X_attention])
X_attention = sum_layer(X_attention)
# compute softmax and get attention weights
X_attention = soft_max_layer(X_attention)
# weighted sum of triplets with attention weights
X_triplets = weighted_sum_layer([ X_triplets, X_attention])

X_concatenated = Concatenate(name="SentenceTripletConcat")([X_question, X_triplets])                      # (None,100)
X_concatenated = Dense(64, activation='relu', name="Dense64")(X_concatenated) # (None,100) -> (None,64)        
Y_pred = Dense(1, activation='sigmoid', name="Dense1")(X_concatenated)        # (None,64)  -> sigmoid    

model = Model(inputs=[question_vectors_input, question_masks_input, triplets_input], outputs=Y_pred)
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
question_vectors_input (InputLa (None, 60, 300)      0                                            
__________________________________________________________________________________________________
Sentence_GRU1 (GRU)             (None, 60, 50)       52650       question_vectors_input[0][0]     
__________________________________________________________________________________________________
triplets_input (InputLayer)     (None, 500, 900)     0                                            
__________________________________________________________________________________________________
question_masks_input (InputLaye (None, 60, 50)       0                                            
__________________________________________________________________________________________________
Sentence_G

In [32]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
#model.fit([question_vectors, question_masks, triplet_vectors], Y, epochs = 30, batch_size = 20)

In [33]:
#TRAIN
s_NumEpocs        = 5
s_NumFilesInBatch = 5
file_list         = glob.glob('X:/cs230/proj0/ned-graphs/data_wiki_encoded/train/train_*.ipwk')
file_list_count   = len(file_list)
file_batch_count  = math.ceil(( file_list_count / s_NumFilesInBatch ))

print(file_list_count)
print(file_batch_count)


459
92


In [ ]:
for ep in range(0,s_NumEpocs):
    if ep != 0:
        file_name = "0
        " + str(ep-1) + ".save"
        model.load_weights(file_name)
    
    file_index       = 0
    batch_file_index = 0
    
    data = []
    # train on batches of files
    for file_index in range(0, file_list_count) :
        inputFile = file_list[file_index]
        file_index       += 1
        batch_file_index += 1
        # print(inputFile)
        f = open(inputFile, 'rb')
        test = pickle.load(f)
        data = np.append(data, test)
        f.close()
    
        if batch_file_index == s_NumFilesInBatch or file_index == file_list_count:
            print("Training on Epoch ", str(ep), "batch ", str(batch_file_index), " Files [" + str(s_NumFilesInBatch) + "] start ", str(file_index - batch_file_index), "/", str(file_list_count) )
            question_vectors, question_masks, triplet_vectors, Y = prepare_data(data, 
                                                                                question_max_words   = s_QuestionMaxWords, 
                                                                                triplets_max_numbers = s_TripletsMaxNumbers, 
                                                                                dimOfQuestionVector  = s_DimOfQuestionVector, 
                                                                                dimOfTripletVector   = s_DimOfTripletVector, 
                                                                                dimOfMask            = s_DimOfMask )
            # train on this batch
            model.fit([question_vectors, question_masks, triplet_vectors], Y, epochs = 10, batch_size = 200, shuffle=True)
            # rest for next batch
            batch_file_index = 0;
            data = []
            
    # save the mode for the epoch
    file_name = "train_epoch_general_attention_" + str(ep) + ".save"
    model.save_weights(file_name)
    



Training on Epoch  0 batch  5  Files [5] start  0 / 459
Epoch 1/10
1000/1000 [==============================] - 19s 19ms/step - loss: 0.6891 - acc: 0.5490
Epoch 2/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.6722 - acc: 0.6510
Epoch 3/10
1000/1000 [==============================] - 13s 13ms/step - loss: 0.6436 - acc: 0.7290
Epoch 4/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.6050 - acc: 0.7610
Epoch 5/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.5549 - acc: 0.7870
Epoch 6/10
1000/1000 [==============================] - 15s 15ms/step - loss: 0.5099 - acc: 0.7970
Epoch 7/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4749 - acc: 0.8160
Epoch 8/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4355 - acc: 0.8120
Epoch 9/10
1000/1000 [==============================] - 14s 14ms/step - loss: 0.4064 - acc: 0.8320
Epoch 10/10
1000/1000 [==============================

1000/1000 [==============================] - 47s 47ms/step - loss: 0.2099 - acc: 0.9060
Epoch 10/10
1000/1000 [==============================] - 48s 48ms/step - loss: 0.2001 - acc: 0.9110
Training on Epoch  0 batch  5  Files [5] start  40 / 459
Epoch 1/10
1000/1000 [==============================] - 44s 44ms/step - loss: 0.4066 - acc: 0.8570
Epoch 2/10
1000/1000 [==============================] - 45s 45ms/step - loss: 0.3530 - acc: 0.8580
Epoch 3/10
1000/1000 [==============================] - 49s 49ms/step - loss: 0.3136 - acc: 0.8720
Epoch 4/10
1000/1000 [==============================] - 48s 48ms/step - loss: 0.3009 - acc: 0.8750
Epoch 5/10
1000/1000 [==============================] - 72s 72ms/step - loss: 0.2730 - acc: 0.8910
Epoch 6/10
1000/1000 [==============================] - 82s 82ms/step - loss: 0.2589 - acc: 0.9010
Epoch 7/10
1000/1000 [==============================] - 67s 67ms/step - loss: 0.2445 - acc: 0.8990
Epoch 8/10
1000/1000 [==============================] - 47s 47

1000/1000 [==============================] - 43s 43ms/step - loss: 0.2860 - acc: 0.8750
Epoch 8/10
1000/1000 [==============================] - 43s 43ms/step - loss: 0.2696 - acc: 0.8830
Epoch 9/10
1000/1000 [==============================] - 43s 43ms/step - loss: 0.2523 - acc: 0.8880
Epoch 10/10
1000/1000 [==============================] - 44s 44ms/step - loss: 0.2324 - acc: 0.8970
Training on Epoch  0 batch  5  Files [5] start  80 / 459
Epoch 1/10
1000/1000 [==============================] - 43s 43ms/step - loss: 0.4751 - acc: 0.8010
Epoch 2/10
1000/1000 [==============================] - 44s 44ms/step - loss: 0.4152 - acc: 0.8170
Epoch 3/10
1000/1000 [==============================] - 44s 44ms/step - loss: 0.3775 - acc: 0.8320
Epoch 4/10
1000/1000 [==============================] - 43s 43ms/step - loss: 0.3512 - acc: 0.8660
Epoch 5/10
1000/1000 [==============================] - 44s 44ms/step - loss: 0.3291 - acc: 0.8600
Epoch 6/10
1000/1000 [==============================] - 44s 44

1000/1000 [==============================] - 49s 49ms/step - loss: 0.2923 - acc: 0.8850
Epoch 6/10
1000/1000 [==============================] - 48s 48ms/step - loss: 0.2778 - acc: 0.8880
Epoch 7/10
1000/1000 [==============================] - 49s 49ms/step - loss: 0.2634 - acc: 0.8910
Epoch 8/10
1000/1000 [==============================] - 48s 48ms/step - loss: 0.2773 - acc: 0.8850
Epoch 9/10
1000/1000 [==============================] - 49s 49ms/step - loss: 0.2643 - acc: 0.8870
Epoch 10/10
1000/1000 [==============================] - 49s 49ms/step - loss: 0.2468 - acc: 0.9000
Training on Epoch  0 batch  5  Files [5] start  120 / 459
Epoch 1/10
1000/1000 [==============================] - 47s 47ms/step - loss: 0.3193 - acc: 0.8580
Epoch 2/10
1000/1000 [==============================] - 48s 48ms/step - loss: 0.3020 - acc: 0.8660
Epoch 3/10
1000/1000 [==============================] - 48s 48ms/step - loss: 0.2847 - acc: 0.8860
Epoch 4/10
1000/1000 [==============================] - 49s 4

1000/1000 [==============================] - 52s 52ms/step - loss: 0.2076 - acc: 0.9080
Epoch 4/10
1000/1000 [==============================] - 52s 52ms/step - loss: 0.1850 - acc: 0.9270
Epoch 5/10
1000/1000 [==============================] - 52s 52ms/step - loss: 0.1791 - acc: 0.9330
Epoch 6/10
1000/1000 [==============================] - 52s 52ms/step - loss: 0.1695 - acc: 0.9420
Epoch 7/10
1000/1000 [==============================] - 52s 52ms/step - loss: 0.1812 - acc: 0.9330
Epoch 8/10
1000/1000 [==============================] - 52s 52ms/step - loss: 0.1772 - acc: 0.9270
Epoch 9/10
1000/1000 [==============================] - 52s 52ms/step - loss: 0.1668 - acc: 0.9330
Epoch 10/10
1000/1000 [==============================] - 52s 52ms/step - loss: 0.1480 - acc: 0.9370
Training on Epoch  0 batch  5  Files [5] start  160 / 459
Epoch 1/10
1000/1000 [==============================] - 49s 49ms/step - loss: 0.3074 - acc: 0.8810
Epoch 2/10
1000/1000 [==============================] - 48s 4

1000/1000 [==============================] - 57s 57ms/step - loss: 0.2811 - acc: 0.8720
Epoch 2/10
1000/1000 [==============================] - 58s 58ms/step - loss: 0.2493 - acc: 0.8830
Epoch 3/10
1000/1000 [==============================] - 58s 58ms/step - loss: 0.2260 - acc: 0.8700
Epoch 4/10
1000/1000 [==============================] - 59s 59ms/step - loss: 0.2089 - acc: 0.8950
Epoch 5/10
1000/1000 [==============================] - 60s 60ms/step - loss: 0.1968 - acc: 0.8880
Epoch 6/10
1000/1000 [==============================] - 59s 59ms/step - loss: 0.1869 - acc: 0.9040
Epoch 7/10
1000/1000 [==============================] - 59s 59ms/step - loss: 0.1770 - acc: 0.9070
Epoch 8/10
1000/1000 [==============================] - 59s 59ms/step - loss: 0.1718 - acc: 0.9140
Epoch 9/10
1000/1000 [==============================] - 59s 59ms/step - loss: 0.1649 - acc: 0.9130
Epoch 10/10
1000/1000 [==============================] - 60s 60ms/step - loss: 0.1569 - acc: 0.9180
Training on Epoch  0

1000/1000 [==============================] - 53s 53ms/step - loss: 0.0869 - acc: 0.9640
Epoch 10/10
1000/1000 [==============================] - 52s 52ms/step - loss: 0.0824 - acc: 0.9660
Training on Epoch  0 batch  5  Files [5] start  235 / 459
Epoch 1/10
1000/1000 [==============================] - 51s 51ms/step - loss: 0.2495 - acc: 0.9000
Epoch 2/10
1000/1000 [==============================] - 50s 50ms/step - loss: 0.3163 - acc: 0.8730
Epoch 3/10
1000/1000 [==============================] - 51s 51ms/step - loss: 0.2134 - acc: 0.8940
Epoch 4/10
1000/1000 [==============================] - 51s 51ms/step - loss: 0.1981 - acc: 0.9030
Epoch 5/10
1000/1000 [==============================] - 51s 51ms/step - loss: 0.1575 - acc: 0.9290
Epoch 6/10
1000/1000 [==============================] - 51s 51ms/step - loss: 0.1639 - acc: 0.9190
Epoch 7/10
1000/1000 [==============================] - 52s 52ms/step - loss: 0.1443 - acc: 0.9350
Epoch 8/10
1000/1000 [==============================] - 51s 5

1000/1000 [==============================] - 65s 65ms/step - loss: 0.1230 - acc: 0.9430
Epoch 8/10
1000/1000 [==============================] - 65s 65ms/step - loss: 0.1173 - acc: 0.9510
Epoch 9/10
1000/1000 [==============================] - 65s 65ms/step - loss: 0.1171 - acc: 0.9470
Epoch 10/10
1000/1000 [==============================] - 66s 66ms/step - loss: 0.1079 - acc: 0.9540
Training on Epoch  0 batch  5  Files [5] start  275 / 459
Epoch 1/10
1000/1000 [==============================] - 57s 57ms/step - loss: 0.2321 - acc: 0.9140
Epoch 2/10
1000/1000 [==============================] - 57s 57ms/step - loss: 0.1823 - acc: 0.9280
Epoch 3/10
1000/1000 [==============================] - 56s 56ms/step - loss: 0.1475 - acc: 0.9430
Epoch 4/10
1000/1000 [==============================] - 56s 56ms/step - loss: 0.1265 - acc: 0.9520
Epoch 5/10
1000/1000 [==============================] - 57s 57ms/step - loss: 0.1180 - acc: 0.9550
Epoch 6/10
1000/1000 [==============================] - 58s 5

In [ ]:
# from keras.layers import Dense, Input, Dropout, LSTM, Activation, SimpleRNN, GRU, Concatenate, Multiply, Reshape, Flatten, Bidirectional

# # layers
# s1_bilstm = Bidirectional(GRU(128, return_sequences = True), name = "s1_bilstm")
# s2_bilstm = Bidirectional(GRU(128, return_sequences = False), name = "s2_bilstm")
# t_bilstm = Bidirectional(GRU(128, return_sequences = False), name = "t_bilstm")
# embedding_layer = pretrained_embedding_layer(word_to_vec_map, words_to_index)
# unstack_layer = Lambda(lambda xin: myUnstack(xin), name="AUnstackLayer")

# # sentence inputs
# sentence_indices = Input(shape=(maxWordsPerSentence,), dtype='int32', name="sentence_indices")   
# sentence_masks = Input(shape=(2*lstm_dim,), dtype='float32', name="sentence_masks")

# # sentence embeddings
# sentence_embeddings = embedding_layer(sentence_indices)       
# # X_sentence = Reshape((maxWordsPerSentence, -1), name="s_reshape")(sentence_embeddings)
# X_sentence = s1_bilstm(sentence_embeddings)
# X_sentence = Multiply()([sentence_masks, X_sentence]) # (None, 60, 256)
# X_sentence = s2_bilstm(X_sentence)

# # triplets
# triplets_input = Input(shape=(maxTriplets, maxWordsPerSentence,), dtype='int32', name="tripletz") 
# triplet_embeddings = embedding_layer(triplets_input)

# X_triplets = Reshape((maxTriplets, -1))(triplet_embeddings)
# X_triplets = t_bilstm(X_triplets)

# X_concatenated = Concatenate()([X_sentence, X_triplets])
# X_concatenated = Dense(256, activation='relu', name="Dense256")(X_concatenated)
# X_concatenated = Dense(64, activation='relu', name="Dense64")(X_concatenated)
# Y_pred = Dense(1, activation='sigmoid', name="Dense1")(X_concatenated)

# m.v2 = Model(inputs=[sentence_indices, sentence_masks, triplets_input], outputs=X_concatenated)
# print(m.v2.summary())